This code provides a foundation for the Sobol indices using the Saltelli method. Pleae make edits where noted

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.stats import t

# Define the spring model as an ODE system
def spring_model(t, y, params):
    C, K, F0, omegaF = params
    z, v = y
    accel_eq = -C * v - K * z + F0 * np.cos(omegaF * t)
    return [v, accel_eq]

# Function to call the spring model and solve it
def call_spring_model(params, IC, tspace):
    sol = solve_ivp(
        spring_model, [tspace[0], tspace[-1]], IC, t_eval=tspace, args=(params,)
    )
    return sol.y[0]

# Cost function for OLS
def get_spring_cost(params, tspace, data, IC):
    sim = call_spring_model(params, IC, tspace)
    residual = data - sim
    return np.sum(residual**2)

# STUDENTS: Define initial conditions, model parameters, and noise variance
IC = []  # Initial conditions
C, K, F0, omegaF = [], [], [], []  # Placeholder values for parameters
noise_std = []  # Noise variance

# STUDENTS: Define true parameters and an initial guess
param_star = []  # True parameters
param_guess = []  # Initial guess

# STUDENTS: Define time range
n_data = []  # Number of data points
tend = []  # Final time
tspace = np.linspace(0, tend, n_data)

# Simulate true data and noisy data
true_data = call_spring_model(param_star, IC, tspace)
noisy_data = true_data + np.random.normal(0, noise_std, len(tspace))
initial_guess = call_spring_model(param_guess, IC, tspace)


# Optimize using OLS
result = minimize(get_spring_cost, param_guess, args=(tspace, noisy_data, IC))
param_opt = result.x
print("Final estimate:", param_opt)
print("True value:", param_star)

# Plot true data, noisy data, and initial guess
plt.figure()
plt.plot(tspace, true_data, '-k', linewidth=2, label='True Data')
plt.plot(tspace, noisy_data, 'ko', markersize=8, label='Noisy Data')
plt.plot(tspace, initial_guess, '--r', linewidth=2, label='Initial Guess')
plt.plot(
    tspace,
    call_spring_model(param_opt, IC, tspace),
    '--b',
    linewidth=2,
    label='Optimized Model',
)
plt.legend()
plt.show()

# Construct confidence and prediction intervals
num_param = 4  # Number of parameters
n_xpts = len(tspace)
S = np.zeros((n_xpts, num_param))
h = 1e-8

for i in range(num_param):
    param_step = param_opt.copy()
    param_step[i] += h
    S[:, i] = (
        call_spring_model(param_step, IC, tspace)
        - call_spring_model(param_opt, IC, tspace)
    ) / h

residual = noisy_data - call_spring_model(param_opt, IC, tspace)
s2 = np.sum(residual**2) / (n_xpts - num_param)
covar = s2 * np.linalg.inv(S.T @ S)

# Confidence intervals
CI_plus = param_opt + t.ppf(0.975, n_xpts - num_param) * np.sqrt(np.diag(covar))
CI_minus = param_opt - t.ppf(0.975, n_xpts - num_param) * np.sqrt(np.diag(covar))
print("Parameter Confidence Intervals:")
print(np.vstack([CI_minus, CI_plus]).T)

# Response confidence and prediction intervals
t_test = np.linspace(0, 1.5 * max(tspace), 100)
n_xtest = len(t_test)
g = np.zeros((n_xtest, num_param))

for i in range(num_param):
    param_step = param_opt.copy()
    param_step[i] += h
    g[:, i] = (
        call_spring_model(param_step, IC, t_test)
        - call_spring_model(param_opt, IC, t_test)
    ) / h

Y_pred = call_spring_model(param_opt, IC, t_test)
Y_CI = np.zeros((n_xtest, 2))
Y_PI = np.zeros((n_xtest, 2))

for i in range(n_xtest):
    y_stderr_CI = t.ppf(0.975, n_xpts - num_param) * np.sqrt(
        g[i, :] @ covar @ g[i, :]
    )
    Y_CI[i, 0] = Y_pred[i] + y_stderr_CI
    Y_CI[i, 1] = Y_pred[i] - y_stderr_CI

    y_stderr_PI = t.ppf(0.975, n_xpts - num_param) * np.sqrt(
        s2 + g[i, :] @ covar @ g[i, :]
    )
    Y_PI[i, 0] = Y_pred[i] + y_stderr_PI
    Y_PI[i, 1] = Y_pred[i] - y_stderr_PI

# Plot model response, CI, and PI
plt.figure()
plt.plot(t_test, Y_pred, '-k', linewidth=2, label='Predicted Response')
plt.fill_between(t_test, Y_CI[:, 0], Y_CI[:, 1], color='blue', alpha=0.2, label='CI')
plt.fill_between(t_test, Y_PI[:, 0], Y_PI[:, 1], color='red', alpha=0.2, label='PI')
plt.scatter(tspace, noisy_data, color='black', label='Noisy Data')
plt.plot(
    t_test, call_spring_model(param_star, IC, t_test), '-c', linewidth=2, label='True Model'
)
plt.legend()
plt.grid()
plt.show()

# Nonlinear uncertainty plot
plt.figure()
plt.plot(t_test, Y_CI[:, 0] - Y_pred, ':b', linewidth=2, label='CI')
plt.plot(t_test, Y_PI[:, 0] - Y_pred, ':r', linewidth=2, label='PI')
plt.legend()
plt.grid()
plt.show()
